In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.decomposition import NMF
from sklearn.neural_network import MLPClassifier # C'est notre composant Deep Learning
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Fixer l'aléatoire pour la reproductibilité
random.seed(42)
np.random.seed(42)

print("Librairies importées avec succès.")

Librairies importées avec succès.


In [3]:
# ---  CHARGEMENT DES DONNÉES EXISTANTES ---

# Chargement des fichiers CSV que vous avez générés
# Assurez-vous que les fichiers sont dans le même dossier que votre notebook
users_df = pd.read_csv('../iam_dataset/users.csv')
rights_df = pd.read_csv('../iam_dataset/rights.csv')

# Optionnel : Charger les définitions pour avoir les noms lisibles (pas strictement nécessaire pour le modèle, mais utile pour l'analyse)
applications_df = pd.read_csv('../iam_dataset/applications.csv')
permissions_df = pd.read_csv('../iam_dataset/permissions.csv')

print("--- Rapport de chargement ---")
print(f"Utilisateurs chargés : {len(users_df)}")
print(f"Droits actifs chargés : {len(rights_df)}")

# Affichage d'un aperçu pour vérification
print("\nAperçu des utilisateurs :")
display(users_df.head(3))
print("\nAperçu des droits :")
display(rights_df.head(3))

--- Rapport de chargement ---
Utilisateurs chargés : 5000
Droits actifs chargés : 41329

Aperçu des utilisateurs :


,user_id,first_name,last_name,position,department,location,contract_type,seniority
0,1,Martin,Rodriguez,Payroll Specialist,Department HR,Rouen,Apprentissage,Junior
1,2,Emmanuel,Grégoire,Security Guard,Department Facilities,Nantes,Apprentissage,Junior
2,3,Matthieu,Roy,IT Support,Department IT,Nantes,Apprentissage,Junior



Aperçu des droits :


,user_id,application_id,permission_id
0,1,18,151
1,1,18,148
2,1,21,176


In [ ]:
# --- PRÉPARATION DE LA MATRICE D'INTERACTION ---

# On crée la matrice pivot à partir des données réelles
# Lignes = user_id, Colonnes = permission_id
user_perm_matrix = rights_df.pivot_table(
    index='user_id', 
    columns='permission_id', 
    aggfunc='size', # Compte le nombre d'occurrences (devrait être 1 ou 0)
    fill_value=0    # Met 0 si le droit n'existe pas
)

# Alignement des données
# Il est possible que certains utilisateurs dans 'users.csv' n'aient AUCUN droit dans 'rights.csv'.
# Le pivot table les ignorerait, ce qui décalerait tout.
# On force donc la matrice à avoir exactement les mêmes lignes que le fichier users.csv.
user_perm_matrix = user_perm_matrix.reindex(users_df['user_id'], fill_value=0)

# Vérification de la cohérence
if user_perm_matrix.shape[0] != len(users_df):
    print("⚠️ ALERTE : Le nombre de lignes de la matrice diffère du nombre d'utilisateurs !")
else:
    print(f"✅ Succès : La matrice est alignée avec les utilisateurs.")
    print(f"Dimensions finales : {user_perm_matrix.shape} (Utilisateurs x Permissions uniques)")

✅ Succès : La matrice est alignée avec les utilisateurs.
Dimensions finales : (5000, 251) (Utilisateurs x Permissions uniques)


In [ ]:
# --- APPRENTISSAGE DES RÔLES (NMF) ---

# (Clusters)
n_roles = 5
nmf = NMF(n_components=n_roles, init='random', random_state=42, max_iter=500)

# W représente l'appartenance des utilisateurs aux rôles
W = nmf.fit_transform(user_perm_matrix)

# On attribue à chaque utilisateur son "Rôle Dominant" (celui avec le score le plus élevé)
# Ce 'role_label' devient la CIBLE (Target) que notre Deep Learning devra prédire plus tard
users_df['role_label'] = np.argmax(W, axis=1)

print("Distribution des rôles trouvés :")
print(users_df['role_label'].value_counts().sort_index())

Distribution des rôles trouvés :
role_label
0    1325
1     547
2    1048
3    1324
4     756
Name: count, dtype: int64


In [ ]:
# --- ENCODAGE DES FEATURES ---

# 1. Sélection des colonnes utiles pour la prédiction
# (Vérifié grâce à l'inspection de votre fichier)
features_cols = ['position', 'department', 'location', 'contract_type', 'seniority']
X_raw = users_df[features_cols]
y = users_df['role_label']

# 2. Encodage (One-Hot Encoding)
# On gère l'erreur potentielle de version de scikit-learn
try:
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
except TypeError:
    encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

X_encoded = encoder.fit_transform(X_raw)

# 3. Split Train/Test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

print(f"Données encodées. Chaque utilisateur est représenté par {X_encoded.shape[1]} variables.")

Données encodées. Chaque utilisateur est représenté par 56 variables.


In [ ]:
# --- ENTRAÎNEMENT DU RÉSEAU DE NEURONES ---

# Définition du modèle (MLP)
dl_model = MLPClassifier(hidden_layer_sizes=(64, 32),
                         activation='relu',
                         solver='adam',
                         max_iter=500,
                         random_state=42)

# Entraînement
dl_model.fit(X_train, y_train)

# Validation
acc = dl_model.score(X_test, y_test)
print(f"Précision du modèle : {acc:.2%}")

Précision du modèle : 100.00%


In [8]:
# --- PRÉDICTION POUR UN NOUVEL ARRIVANT ---

# Définissez ici le profil du nouvel arrivant
new_profil = pd.DataFrame([{
    'position': 'DevOps',         # Doit exister dans vos données d'origine
    'department': 'Department IT',# Attention à la casse exacte
    'location': 'Nantes',
    'contract_type': 'CDI',
    'seniority': 'Senior'
}])

# Encodage & Prédiction
new_vector = encoder.transform(new_profil)
pred_role = dl_model.predict(new_vector)[0]
pred_proba = dl_model.predict_proba(new_vector)[0]

confidence = pred_proba[pred_role]

print(f"--- RÉSULTAT ---")
print(f"Pour un {new_profil.iloc[0]['position']} à {new_profil.iloc[0]['location']} :")
print(f"Rôle Suggéré : {pred_role}")
print(f"Confiance IA : {confidence:.2%}")

if confidence > 0.8:
    print("✅ Confiance élevée -> Attribution automatique possible.")
else:
    print("⚠️ Confiance faible -> Demander validation humaine (RLHF).")

--- RÉSULTAT ---
Pour un DevOps à Nantes :
Rôle Suggéré : 0
Confiance IA : 99.97%
✅ Confiance élevée -> Attribution automatique possible.
